# MDI343
## Lab on real-world graph analysis -- clustering

The objective of this lab is to get a feeling of real-world graphs. For information on the `scikit-network` library, [the documentation is handy](https://scikit-network.readthedocs.io/).

## Import

In [1]:
import numpy as np

In [2]:
import networkx as nx

In [3]:
import matplotlib.pyplot as plt
from itertools import groupby

In [4]:
import sknetwork as skn

In [5]:
# Util function to plot the inverse cumulative distribution
def ccdf(values):
    x = []
    y = []
    values = sorted(values)

    # First make dist
    dist = [(key, len(list(group))) for key, group in groupby(values)]

    # Then compute inverse cumulative
    total = 1.0
    for (val, count) in dist:
        x.append(val)
        y.append(total)
        total -= count/len(values)
    return x, y

# Util function to return the distribution of values
def dist(values):
    values = sorted(values)

    # First make dist
    dist = [(key, len(list(group))) for key, group in groupby(values)]
    
    return [x[0] for x in dist], [x[1] for x in dist]

## Load data

We will work on 2 graphs induced by the [Vital articles of Wikipedia](https://en.wikipedia.org/wiki/Wikipedia:Vital_articles/Level/4), a selection of about 10,000 articles of the English Wikipedia:
* the directed graph of hyperlinks between these articles,
* the bipartite graph between articles and (stemmed) words used in their summary.

In [6]:
data = skn.data.load_netset('wikivitals')
data.keys()

dict_keys(['names_col', 'adjacency', 'names', 'labels', 'names_row', 'names_labels_hierarchy', 'meta', 'names_labels', 'labels_hierarchy', 'biadjacency'])

In [10]:
# graph of links
adjacency = data.adjacency
adjacency

<10012x10012 sparse matrix of type '<class 'numpy.bool_'>'
	with 792091 stored elements in Compressed Sparse Row format>

In [9]:
# graph of words
biadjacency = data.biadjacency
biadjacency

<10012x31323 sparse matrix of type '<class 'numpy.int64'>'
	with 995919 stored elements in Compressed Sparse Row format>

In [11]:
# article names
names = data.names

In [12]:
# article categories
categories = data.names_labels
categories

array(['People', 'History', 'Geography', 'Arts',
       'Philosophy and religion', 'Everyday life',
       'Society and social sciences', 'Biology and health sciences',
       'Physical sciences', 'Technology', 'Mathematics'], dtype='<U27')

In [13]:
# words
words = data.names_col
words

array(['moos', 'tonnag', 'separatist', ..., 'luteum', 'radiat', 'helena'],
      dtype='<U22')

In [14]:
node_index = {name:i for i, name in enumerate(names)}

In [15]:
n_articles, n_words = biadjacency.shape

In [16]:
labels = data.labels

## To do

For the 2 graphs:
* Cluster the graph.
* Compute some statistics about the clustering: how many clusters are there? Are they balanced (in number of nodes? in number of edges?)
* Using PageRank, display the 5 most important nodes of each cluster.
* Evaluate the quality of the clustering using the categories as ground-truth.
* What does the ARI score mean? Do we have a good clustering? How can we know?

In [52]:
from sklearn.metrics import adjusted_rand_score as ari

In [53]:
louvain = skn.clustering.Louvain()

In [54]:
louvain.fit(adjacency)

Louvain(resolution=1.0, modularity='dugue', tol_aggregation=0.001, n_aggregations=-1, shuffle_nodes=False, sort_clusters=True, return_membership=True, return_aggregate=True)

In [69]:
ari(labels, louvain.labels_)

0.3018954851357631

## A bit of comparison
`scikit-network` also implements a clustering method based on the _$k$-means_ algorithm. As you know, graphs are not metric spaces, and so any method must first project the graph to a metric space, then run $k$-means on this space.

For the 2 graphs:
   * Run a $k$-means clustering.
   * Compute the same base statistics about the clusters (as for Louvain); what can you say?
   * Set up a method to compare the clusters obtained through $k$-means and Louvain. Can you say one is better than the other? In which regard(s)?